In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 6
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [128]  # 单层LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.zeros(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 构建模型
class BiLSTM(object):
    """
    Bi-LSTM 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)

            # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
            # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm, self.embeddedWords, dtype=tf.float32)
        
        # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2]
        concatedOutput = tf.concat(outputs, 2)
        
        # 去除最后时间步的输出作为全连接的输入
        finalOutput = concatedOutput[:, -1, :]
        
        outputSize = config.model.hiddenSizes[-1] * 2  # 因为是双向LSTM，最终的输出值是fw和bw的拼接，因此要乘以2
        output = tf.reshape(finalOutput, [-1, outputSize])  # reshape成全连接层的输入维度
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTM(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0

2019-01-02T16:59:38.574662, step: 50, loss: 0.44961315393447876, acc: 0.8359, auc: 0.8755, precision: 0.8704, recall: 0.7705
2019-01-02T16:59:38.966104, step: 51, loss: 0.48873838782310486, acc: 0.8203, auc: 0.8762, precision: 0.72, recall: 0.9643
2019-01-02T16:59:39.366052, step: 52, loss: 0.6200796365737915, acc: 0.6797, auc: 0.7782, precision: 0.5957, recall: 0.9492
2019-01-02T16:59:39.827507, step: 53, loss: 0.506675124168396, acc: 0.7812, auc: 0.8385, precision: 0.7407, recall: 0.8955
2019-01-02T16:59:40.220504, step: 54, loss: 0.48574212193489075, acc: 0.7969, auc: 0.8354, precision: 0.8, recall: 0.8
2019-01-02T16:59:40.634257, step: 55, loss: 0.5988105535507202, acc: 0.7422, auc: 0.7713, precision: 0.8644, recall: 0.6711
2019-01-02T16:59:41.060946, step: 56, loss: 0.6629310846328735, acc: 0.7031, auc: 0.7243, precision: 0.7414, recall: 0.6515
2019-01-02T16:59:41.482556, step: 57, loss: 0.5766755938529968, acc: 0.75, auc: 0.8087, precision: 0.9048, recall: 0.5758
2019-01-02T16:59

2019-01-02T17:00:21.578079, step: 116, loss: 0.6729180216789246, acc: 0.5625, auc: 0.5586, precision: 0.8333, recall: 0.1562
2019-01-02T17:00:21.986325, step: 117, loss: 0.6651807427406311, acc: 0.5703, auc: 0.5597, precision: 0.9, recall: 0.1429
2019-01-02T17:00:22.360390, step: 118, loss: 0.6686782836914062, acc: 0.6172, auc: 0.5331, precision: 0.8667, recall: 0.2167
2019-01-02T17:00:22.784244, step: 119, loss: 0.6412840485572815, acc: 0.5312, auc: 0.6424, precision: 1.0, recall: 0.1667
2019-01-02T17:00:23.160363, step: 120, loss: 0.6676203608512878, acc: 0.5625, auc: 0.5994, precision: 0.9, recall: 0.1406
2019-01-02T17:00:23.570971, step: 121, loss: 0.6119674444198608, acc: 0.5938, auc: 0.6863, precision: 1.0, recall: 0.2464
2019-01-02T17:00:23.983651, step: 122, loss: 0.6208840608596802, acc: 0.625, auc: 0.6862, precision: 0.9474, recall: 0.2769
2019-01-02T17:00:24.364589, step: 123, loss: 0.640465259552002, acc: 0.5391, auc: 0.621, precision: 1.0, recall: 0.169
2019-01-02T17:00:24

2019-01-02T17:00:49.727665, step: 184, loss: 0.6820317506790161, acc: 0.5234, auc: 0.5832, precision: 1.0, recall: 0.0317
2019-01-02T17:00:50.182810, step: 185, loss: 0.6797091960906982, acc: 0.5312, auc: 0.6172, precision: 0.0, recall: 0.0
2019-01-02T17:00:50.614497, step: 186, loss: 0.6912664771080017, acc: 0.5625, auc: 0.5365, precision: 1.0, recall: 0.0345
2019-01-02T17:00:51.026541, step: 187, loss: 0.6833398342132568, acc: 0.4531, auc: 0.5766, precision: 0.0, recall: 0.0
2019-01-02T17:00:51.451925, step: 188, loss: 0.6832302212715149, acc: 0.4453, auc: 0.5473, precision: 0.0, recall: 0.0
2019-01-02T17:00:51.860308, step: 189, loss: 0.679970920085907, acc: 0.5859, auc: 0.6212, precision: 1.0, recall: 0.0364
2019-01-02T17:00:52.230247, step: 190, loss: 0.6605371236801147, acc: 0.4688, auc: 0.6751, precision: 1.0, recall: 0.0286
2019-01-02T17:00:52.607274, step: 191, loss: 0.7064157724380493, acc: 0.5938, auc: 0.5104, precision: 0.5, recall: 0.0192
2019-01-02T17:00:53.000515, step: 

2019-01-02T17:01:33.006978, step: 250, loss: 0.8092740774154663, acc: 0.4766, auc: 0.5998, precision: 1.0, recall: 0.0147
2019-01-02T17:01:33.428436, step: 251, loss: 0.8290076851844788, acc: 0.4297, auc: 0.5096, precision: 0.0, recall: 0.0
2019-01-02T17:01:33.845774, step: 252, loss: 0.7447402477264404, acc: 0.5078, auc: 0.5614, precision: 1.0, recall: 0.0308
2019-01-02T17:01:34.291230, step: 253, loss: 0.7368309497833252, acc: 0.5078, auc: 0.5213, precision: 0.0, recall: 0.0
2019-01-02T17:01:34.710316, step: 254, loss: 0.7568743228912354, acc: 0.4766, auc: 0.4614, precision: 0.3333, recall: 0.0152
2019-01-02T17:01:35.127427, step: 255, loss: 0.7142102122306824, acc: 0.5156, auc: 0.4923, precision: 0.6, recall: 0.0476
2019-01-02T17:01:35.530699, step: 256, loss: 0.7115918397903442, acc: 0.5234, auc: 0.4969, precision: 0.5, recall: 0.0328
2019-01-02T17:01:35.936360, step: 257, loss: 0.7096775770187378, acc: 0.4062, auc: 0.5195, precision: 0.6667, recall: 0.0759
2019-01-02T17:01:36.3481

...

2019-01-02T17:02:42.578669, step: 382, loss: 0.6697988510131836, acc: 0.5547, auc: 0.5924, precision: 0.8462, recall: 0.1667
2019-01-02T17:02:42.959389, step: 383, loss: 0.6898916959762573, acc: 0.5156, auc: 0.5268, precision: 1.0, recall: 0.0746
2019-01-02T17:02:43.341593, step: 384, loss: 0.6813129186630249, acc: 0.5625, auc: 0.5238, precision: 0.75, recall: 0.1
2019-01-02T17:02:43.728311, step: 385, loss: 0.6623761057853699, acc: 0.5469, auc: 0.6074, precision: 1.0, recall: 0.0938
2019-01-02T17:02:44.111627, step: 386, loss: 0.6553503274917603, acc: 0.5703, auc: 0.6244, precision: 0.875, recall: 0.1148
2019-01-02T17:02:44.499519, step: 387, loss: 0.6671549677848816, acc: 0.5469, auc: 0.5708, precision: 1.0, recall: 0.0938
2019-01-02T17:02:44.912148, step: 388, loss: 0.655626654624939, acc: 0.625, auc: 0.5745, precision: 0.8, recall: 0.0784
2019-01-02T17:02:45.317024, step: 389, loss: 0.6839375495910645, acc: 0.5625, auc: 0.5214, precision: 0.6667, recall: 0.0351
2019-01-02T17:02:45.

2019-01-02T17:03:25.071058, step: 448, loss: 0.6355047225952148, acc: 0.5547, auc: 0.6703, precision: 0.8, recall: 0.1818
2019-01-02T17:03:25.462673, step: 449, loss: 0.661806583404541, acc: 0.5781, auc: 0.5726, precision: 0.7273, recall: 0.1356
2019-01-02T17:03:25.877162, step: 450, loss: 0.6549288630485535, acc: 0.5938, auc: 0.5855, precision: 0.9, recall: 0.15
2019-01-02T17:03:26.317839, step: 451, loss: 0.6513785123825073, acc: 0.625, auc: 0.5886, precision: 0.875, recall: 0.1296
2019-01-02T17:03:26.721265, step: 452, loss: 0.7000529766082764, acc: 0.4297, auc: 0.5931, precision: 0.7143, recall: 0.0658
2019-01-02T17:03:27.102502, step: 453, loss: 0.6909462809562683, acc: 0.4375, auc: 0.588, precision: 0.6667, recall: 0.0274
2019-01-02T17:03:27.482908, step: 454, loss: 0.6493810415267944, acc: 0.5625, auc: 0.6144, precision: 0.8889, recall: 0.127
2019-01-02T17:03:27.907136, step: 455, loss: 0.6854274868965149, acc: 0.5469, auc: 0.5639, precision: 0.7273, recall: 0.127
2019-01-02T17:

2019-01-02T17:04:07.055249, step: 513, loss: 0.6634961366653442, acc: 0.5625, auc: 0.6163, precision: 0.7143, recall: 0.1613
2019-01-02T17:04:07.472477, step: 514, loss: 0.6541314721107483, acc: 0.5781, auc: 0.5623, precision: 0.8333, recall: 0.2273
2019-01-02T17:04:07.913293, step: 515, loss: 0.6484103202819824, acc: 0.5312, auc: 0.616, precision: 0.8125, recall: 0.1857
2019-01-02T17:04:08.364946, step: 516, loss: 0.6800132989883423, acc: 0.5625, auc: 0.551, precision: 0.6923, recall: 0.1475
2019-01-02T17:04:08.794072, step: 517, loss: 0.6626937985420227, acc: 0.5625, auc: 0.5961, precision: 0.6667, recall: 0.069
2019-01-02T17:04:09.193845, step: 518, loss: 0.6466397047042847, acc: 0.5391, auc: 0.5623, precision: 0.9, recall: 0.1343
2019-01-02T17:04:09.598953, step: 519, loss: 0.6786009073257446, acc: 0.5312, auc: 0.5553, precision: 0.6667, recall: 0.0952
2019-01-02T17:04:10.009889, step: 520, loss: 0.6652137041091919, acc: 0.5078, auc: 0.6004, precision: 0.7, recall: 0.1045
2019-01-0

2019-01-02T17:04:34.382851, step: 580, loss: 0.6683982610702515, acc: 0.5703, auc: 0.5702, precision: 0.6667, recall: 0.1034
2019-01-02T17:04:34.819653, step: 581, loss: 0.6807287335395813, acc: 0.5312, auc: 0.546, precision: 0.6667, recall: 0.1538
2019-01-02T17:04:35.202826, step: 582, loss: 0.6380206942558289, acc: 0.5703, auc: 0.5899, precision: 1.0, recall: 0.1791
2019-01-02T17:04:35.605509, step: 583, loss: 0.6494115591049194, acc: 0.5859, auc: 0.6504, precision: 0.9, recall: 0.1475
2019-01-02T17:04:36.021379, step: 584, loss: 0.6834056973457336, acc: 0.5469, auc: 0.5978, precision: 0.6667, recall: 0.129
2019-01-02T17:04:36.410965, step: 585, loss: 0.6355092525482178, acc: 0.5547, auc: 0.6056, precision: 0.8, recall: 0.0667
2019-01-02T17:04:36.794250, step: 586, loss: 0.6619155406951904, acc: 0.5938, auc: 0.5883, precision: 0.8571, recall: 0.1053
2019-01-02T17:04:37.196180, step: 587, loss: 0.6471589803695679, acc: 0.625, auc: 0.6394, precision: 0.7692, recall: 0.1818
2019-01-02T1

2019-01-02T17:05:17.290401, step: 645, loss: 0.6177353858947754, acc: 0.6094, auc: 0.6805, precision: 0.8333, recall: 0.1724
